In [1]:
import esm
import pyvista as pv
import os
import pandas as pd
import numpy as np
import trimesh as tm

cwd = os.getcwd()
src = os.path.dirname(cwd)

Path to radiance is set to: /usr/local/radiance
Path to perl is set to: /usr/bin


In [2]:
# import all external geometry with pyvista
wal_filepath = os.path.join(src, "data", "test_room_walls_02.obj")
wal_mesh = pv.read(wal_filepath)
fen_filepath = os.path.join(src, "data", "test_room_fens_02.obj")
fen_mesh = pv.read(fen_filepath)

con_buildings_filepath = os.path.join(src, "data", "test_context_01_buildings.obj")
con_buildings_mesh = pv.read(con_buildings_filepath)
con_ground_filepath = os.path.join(src, "data", "test_context_01_ground.obj")
con_ground_mesh = pv.read(con_ground_filepath)
# con_sky_filepath = os.path.join(src, "data", "test_context_01_sky.obj")
# con_sky_mesh = pv.read(con_sky_filepath)
con_vegetation_filepath = os.path.join(src, "data", "test_context_01_vegetation.obj")
con_vegetation_mesh = pv.read(con_vegetation_filepath)

# construct honeybee surfaces from the mesh
wal_hb_surf = esm.pv_mesh_to_hbsurface(wal_mesh, 0, "wall", esm.material_plastic)
fen_hb_surf = esm.pv_mesh_to_hbsurface(fen_mesh, 5, "window", esm.material_glass)
con_buildings_hb_surf = esm.pv_mesh_to_hbsurface(con_buildings_mesh, 0, "context_buildings", esm.material_plastic)
con_ground_hb_surf = esm.pv_mesh_to_hbsurface(con_ground_mesh, 0, "context_ground", esm.material_plastic)
# con_sky_hb_surf = esm.pv_mesh_to_hbsurface(con_sky_mesh, 0, "context_sky", esm.material_plastic)
con_vegetation_hb_surf = esm.pv_mesh_to_hbsurface(con_vegetation_mesh, 0, "context_vegetation", esm.material_plastic)

hb_surfaces = wal_hb_surf + fen_hb_surf + con_buildings_hb_surf + con_ground_hb_surf + con_vegetation_hb_surf

In [3]:
# generate grid of test points
test_pts = [(1, 1, 1),(6, 1, 1),(1, 6, 1),(1, 1, 6)]
test_vec = [(0, 0, 1),(0, 0, 1),(0, 0, 1),(0, 0, 1)]
analysis_grid = esm.AnalysisGrid.from_points_and_vectors(test_pts, test_vec, 'test_grid')

# put the recipe together
rp = esm.ContextViewGridBased(analysis_grids=(analysis_grid,),
               hb_objects=hb_surfaces)

Found 5486 opaque surfaces.
Found 1 fenestration surfaces.
Found 0 window-groups.


In [4]:
# write simulation to folder
project_name = 'clean_context_view'
batch_file = rp.write(
    target_folder='.', project_name=project_name)


Writing recipe contents to: ./clean_context_view/gridbased


In [5]:
# run the simulation
rp.run(batch_file, debug=False)

True

In [7]:
# load rtrace results
rs_path = os.path.join(project_name, 'gridbased', 'rtrace_res.txt')
rtrace_res = pd.read_csv(rs_path, skiprows=8, sep='\t', usecols=[3,4,5], header=None, names=['last_int_name', 'first_int_dist', 'last_int_dist'])

# load the rays
rays_path = os.path.join(project_name, 'gridbased', 'view_rays.txt')
rays_df = pd.read_csv(rays_path, skiprows=0, sep=' ', usecols=[3,4,5], header=None, names=['Rx', 'Ry', 'Rz'])

# concatenate data frames
results = pd.concat([rays_df, rtrace_res], axis = 1)
results

,Rx,Ry,Rz,last_int_name,first_int_dist,last_int_dist
0,-0.525731,0.850651,0.000000,wall,1.902113,1.902113
1,0.525731,0.850651,0.000000,context_buildings,4.205226,4.205226
2,-0.525731,-0.850651,0.000000,wall,1.175571,1.175571
3,0.525731,-0.850651,0.000000,window,119.185700,1.175571
4,0.000000,-0.525731,0.850651,wall,1.902113,1.902113
...,...,...,...,...,...,...
10237,-0.953549,-0.300528,0.020669,wall,1.048714,1.048714
10238,-0.939160,-0.333520,0.082120,wall,1.064782,1.064782
10239,-0.929648,-0.367868,0.020669,wall,1.075676,1.075676
10240,-0.951932,-0.300080,0.061453,wall,1.050495,1.050495


In [8]:
window_mask = (results['last_int_dist'] - results['first_int_dist']).abs() > 0
print('window view solid angle:',4 * np.pi * window_mask.sum()/len(window_mask), 'steradian')

window view solid angle: 1.9459347582868236 steradian


In [10]:
horizontal_line_mask = results['Rz'].abs() < 0.01
print('horizontal window view angle:',(2 * np.pi * window_mask * horizontal_line_mask).sum() / horizontal_line_mask.sum(), 'radians')

horizontal window view angle: 2.012582793705961 radians


In [9]:
view_quality_ratio = results['last_int_name'].value_counts() / len(results)
view_quality_ratio

wall                 0.553310
context_ground       0.257762
window               0.154853
context_buildings    0.034075
Name: last_int_name, dtype: float64

In [ ]:
p = pv.Plotter(notebook=True)

icosphere = tm.creation.icosphere(subdivisions=5, radius=1.0)

p.add_mesh(icosphere.vertices, scalars=results['last_int_dist'], point_size=10)

p.add_axes()

p.show(use_ipyvtk=True)